In [ ]:
# Smart Mirror Project Summary

## Project Overview

This notebook summarizes the discussions and plan for the Smart Mirror project as of April 28, 2025.

## Current Project Status

- **Hardware**: Raspberry Pi 5 (8GB RAM), Camera (ov5647), Hailo-8 AI Accelerator
- **Software**: Python with PyQt5 for GUI, virtual environment (smart_mirror_venv)
- **Working Components**: 
  - Basic camera interface (`camera_interface.py`)
  - Partial face detection implementation (`hailo_face_detector.py`)
  - Simple GUI with time display (`pyqt_gui_with_camera.py`)
  - Main app gui vs 1 : /home/taran/smart_mirror_project/pyqt_gui/pyqt_gui_camera_and_hailo.py
  - Main app gui vs 2 : /home/taran/smart_mirror_project/pyqt_gui/Main_App_Launch.py

    Unsure which one to use. Still confused about hailort vs pyhailort

## Project Structure

```
smart_mirror_project/
├── data/
│   ├── daily_captures/
│   ├── processed/
│   ├── raw/
│   ├── user_profiles/
│   └── weather_data/
├── gui/
├── models/
│   └── hailo/
├── notebooks/
├── pyqt_gui/
│   ├── pyqt_gui_camera_and_hailo.py
│   ├── pyqt_gui_with_camera.py
│   └── smart_mirror_mvp.py (to be created)
├── src/
│   ├── camera/
│   │   └── camera_interface.py
│   ├── face_detection/
│   │   └── hailo_face_detector.py
│   ├── user_management/
│   │   └── user_profiles.py (to be created)
│   └── weather/
│       └── open_meteo.py (to be created)
└── tests/
```

## Core Requirements

1. **Main App Features**:
   - Face detection using Hailo-8 AI accelerator
   - User registration with profile creation
   - Display time, date, and weather

2. **Weather Display**:
   - Default location: Brasov, Romania (when no user is recognized)
   - Switch to user's preferred location when recognized

3. **User Profiles**:
   - Store in JSON format (simple file-based storage)
   - Include: name, gender, DOB, location (city, state, country)
   - Location data used for personalized weather

4. **Face Recognition**:
   - Detect faces using Hailo-8
   - Match to registered user profiles
   - Option to register and train new faces

## Implementation Plan

### 1. Base UI with Camera Integration

Start with `pyqt_gui_with_camera.py` which already has:
- Basic UI with time display
- Camera initialization and frame capture
- Clean PyQt5 structure

### 2. Add Face Detection

Integrate the Hailo face detector:
```python
# Add to imports
from src.face_detection.hailo_face_detector import HailoFaceDetector

# Add to camera initialization
self.face_detector = HailoFaceDetector()

# Modify process_camera_frame to use face detection
def process_camera_frame(self):
    frame = self.camera.capture_frame()
    if frame is not None:
        # Pass frame to face detector
        faces = self.face_detector.detect_faces(frame)
        
        if len(faces) > 0:
            self.camera_status_label.setText(f"Detected {len(faces)} face(s)")
        else:
            self.camera_status_label.setText("Camera: No faces detected")
    else:
        self.camera_status_label.setText("Camera: Error")
```

### 3. Add Weather Service for Brasov

Create weather service module (`src/weather/open_meteo.py`):
```python
import requests

def get_weather(location="default"):
    """
    Get current weather data for the specified location
    If location is "default", returns weather for Brasov, Romania
    """
    try:
        # Default location: Brasov, Romania
        if location == "default":
            lat = 45.6427
            lon = 25.5887
            location_name = "Brasov, Romania"
        else:
            # For user profiles with custom locations
            lat = location["lat"]
            lon = location["lon"]
            location_name = location["name"]
        
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code,wind_speed_10m&temperature_unit=fahrenheit"
        
        response = requests.get(url)
        data = response.json()
        
        temp = round(data["current"]["temperature_2m"])
        wind = round(data["current"]["wind_speed_10m"])
        
        return {
            "temperature": temp,
            "wind": wind,
            "location": location_name
        }
    except Exception as e:
        print(f"Error getting weather: {e}")
        return {
            "temperature": 50,
            "wind": 5,
            "location": "Brasov, Romania"
        }
```

### 4. User Profile Management

Create user profile module with JSON storage (`src/user_management/user_profiles.py`):
```python
import json
import os

# File path for storing user profiles
PROFILES_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 
                          "data", "user_profiles")

def load_profile(user_id):
    """Load a specific user profile by ID"""
    try:
        profile_path = os.path.join(PROFILES_DIR, f"{user_id}.json")
        if os.path.exists(profile_path):
            with open(profile_path, 'r') as file:
                return json.load(file)
        return None
    except Exception as e:
        print(f"Error loading profile: {e}")
        return None

def save_profile(user_profile):
    """Save a user profile"""
    try:
        # Create profiles directory if it doesn't exist
        os.makedirs(PROFILES_DIR, exist_ok=True)
        
        # Get user ID from profile
        user_id = user_profile["id"]
        
        # Save profile
        profile_path = os.path.join(PROFILES_DIR, f"{user_id}.json")
        with open(profile_path, 'w') as file:
            json.dump(user_profile, file, indent=4)
        return True
    except Exception as e:
        print(f"Error saving profile: {e}")
        return False

def get_all_profiles():
    """Get all user profiles"""
    profiles = {}
    try:
        if os.path.exists(PROFILES_DIR):
            for filename in os.listdir(PROFILES_DIR):
                if filename.endswith(".json"):
                    profile_path = os.path.join(PROFILES_DIR, filename)
                    with open(profile_path, 'r') as file:
                        profile = json.load(file)
                        profiles[profile["id"]] = profile
    except Exception as e:
        print(f"Error loading profiles: {e}")
    return profiles

def get_default_user():
    """Get the default user (no user recognized)"""
    return {
        "id": "default",
        "name": "Guest",
        "location": "default"  # This will use Brasov, Romania
    }
```

### 5. User Registration Interface

Add registration functionality to the main app:

```python
def show_registration_form(self):
    """Show user registration form"""
    from PyQt5.QtWidgets import (QDialog, QFormLayout, QLineEdit, 
                               QPushButton, QComboBox, QDateEdit)
    import uuid
    
    dialog = QDialog(self)
    dialog.setWindowTitle("User Registration")
    layout = QFormLayout()
    
    # First Name field
    first_name = QLineEdit()
    layout.addRow("First Name:", first_name)
    
    # Gender selection
    gender = QComboBox()
    gender.addItems(["Male", "Female", "Other"])
    layout.addRow("Gender:", gender)
    
    # Date of Birth
    dob = QDateEdit()
    layout.addRow("Date of Birth:", dob)
    
    # Location fields
    city = QLineEdit()
    layout.addRow("City:", city)
    
    country = QLineEdit()
    layout.addRow("Country:", country)
    
    state = QLineEdit()
    state.setPlaceholderText("Only if USA")
    layout.addRow("State:", state)
    
    # Submit button
    submit_btn = QPushButton("Register")
    layout.addRow(submit_btn)
    
    dialog.setLayout(layout)
    
    # Connect submit button
    def register():
        # Generate unique ID
        user_id = str(uuid.uuid4())
        
        # Create location name
        location_name = f"{city.text()}, {state.text() if state.text() and country.text().lower() == 'usa' else ''} {country.text()}"
        
        # Create user profile
        user_profile = {
            "id": user_id,
            "name": first_name.text(),
            "gender": gender.currentText(),
            "dob": dob.date().toString("yyyy-MM-dd"),
            "location": {
                "name": location_name,
                "city": city.text(),
                "state": state.text(),
                "country": country.text(),
                "lat": 0,  # This would need geocoding in a real app
                "lon": 0   # This would need geocoding in a real app
            }
        }
        
        # Save profile
        from src.user_management.user_profiles import save_profile
        save_profile(user_profile)
        
        # Train facial recognition for this user
        self.train_face_for_user(user_id)
        
        dialog.accept()
    
    submit_btn.clicked.connect(register)
    dialog.exec_()

def train_face_for_user(self, user_id):
    """Capture and save face data for a new user"""
    # This would capture multiple frames, detect the face,
    # and save face encodings for future recognition
    pass
```

### 6. Complete MVP Implementation

The final MVP should:
1. Display time, date, weather for Brasov by default
2. Detect faces using the Hailo accelerator
3. Match detected faces to user profiles
4. Show personalized weather and greetings when users are recognized
5. Provide a user registration interface

## Next Steps

- Implement the weather module first
- Add user profile storage
- Integrate face detection with the UI
- Add user registration interface
- Test the complete system

## Future Enhancements

- Add more widgets (calendar, news, etc.)
- Improve face recognition accuracy
- Add voice commands
- Create admin interface for managing users
- Add customizable layouts and themes